In [87]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster
import difflib
from shapely.geometry import Point

In [91]:
df = pd.read_excel("Cholera.xlsx")

,State,LGA,ward
0,Abia,Umuahia South,Ubakala A Ward
1,Ondo,Okitipupa,OkitipupaI 1 Ward
2,Ondo,Okitipupa,OkitipupaI 1 Ward
3,Ebonyi,Ishielu,NaN
4,Ebonyi,Ishielu,NaN
...,...,...,...
22776,Jigawa,Birniwa,Birniwa Ward
22777,Yobe,Gulani,Bumsa Ward
22778,Yobe,Gulani,Bularafa Ward
22779,Adamawa,Shelleng,Shelleng Ward


In [129]:
df3 = gpd.read_file("nigerian-public-water-supplies.json")

In [268]:
df4 = gpd.read_file('nigerian-dump-sites.json')

In [83]:
df5 = gpd.read_file('/Users/macbookair/Downloads/EHA TASKS/EHA Tasks 2/NORTHWEST INDICATORS/GRID3_Nigeria_-_Local_Government_Area_Boundaries/GRID3_Nigeria_-_Local_Government_Area_Boundaries.shp')

In [84]:
df6 = gpd.read_file('/Users/macbookair/Downloads/nigerian-markets/nigerian-markets.json')

In [85]:
df7 = gpd.read_file('/Users/macbookair/Downloads/EHA TASKS/EHA Tasks 2/NORTHWEST INDICATORS/GRID3_Nigeria_-_Ward_Boundaries/GRID3_Nigeria_-_Ward_Boundaries.shp')

In [423]:
df5['Area'] = df5['geometry'].apply(lambda x:x.area)

In [396]:
df3= df3.to_crs('epsg:26332')

In [398]:
df4 = df4.to_crs('epsg:26332')

In [404]:
df5=df5.to_crs('epsg:26332')

In [465]:
df7 = df7.to_crs('epsg:26332')

In [93]:
df8 = df[~df['ward'].isna()]

In [94]:
df8

,State,LGA,ward
0,Abia,Umuahia South,Ubakala A Ward
1,Ondo,Okitipupa,OkitipupaI 1 Ward
2,Ondo,Okitipupa,OkitipupaI 1 Ward
167,Ebonyi,Ohaukwu,Ngbo Ward
168,Ebonyi,Ohaukwu,Ngbo Ward
...,...,...,...
22776,Jigawa,Birniwa,Birniwa Ward
22777,Yobe,Gulani,Bumsa Ward
22778,Yobe,Gulani,Bularafa Ward
22779,Adamawa,Shelleng,Shelleng Ward


In [104]:
df8['ward'] = df8['ward'].apply(lambda x:" ".join(x.split(" ")[:-1]))

/var/folders/ks/cpcnjkt95qx6v7hynw8xydkr0000gp/T/ipykernel_3431/3286625278.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df8['ward'] = df8['ward'].apply(lambda x:" ".join(x.split(" ")[:-1]))


In [105]:
df8

,State,LGA,ward
0,Abia,Umuahia South,Ubakala A
1,Ondo,Okitipupa,OkitipupaI 1
2,Ondo,Okitipupa,OkitipupaI 1
167,Ebonyi,Ohaukwu,Ngbo
168,Ebonyi,Ohaukwu,Ngbo
...,...,...,...
22776,Jigawa,Birniwa,Birniwa
22777,Yobe,Gulani,Bumsa
22778,Yobe,Gulani,Bularafa
22779,Adamawa,Shelleng,Shelleng


In [106]:
df8 = df8.reset_index(drop=True)

In [107]:
df8 = df8.drop_duplicates()

In [108]:
df8['ward'] =df8['ward'].apply(lambda x:x.replace('Ward', ''))

In [109]:
df8 = df8.reset_index(drop=True)

In [110]:
df8 

,State,LGA,ward
0,Abia,Umuahia South,Ubakala A
1,Ondo,Okitipupa,OkitipupaI 1
2,Ebonyi,Ohaukwu,Ngbo
3,Bayelsa,Southern Ijaw,Ekowe
4,Ondo,Irele,Akotogbo
...,...,...,...
904,Benue,Agatu,Usha
905,Zamfara,Maradun,Damaga/Gama Giwa
906,Benue,Agatu,Egba
907,Adamawa,Girei,Girei A


In [112]:
def get_ward_code(data1, data2):
    cholera = data1.copy()
    cholera['WardCode'] = ""
    cholera['geometry'] = ""
    size = cholera.shape[0]
    for i in range(0,size):
        ward = cholera.loc[i,'ward']
        lga = cholera.loc[i,'LGA']
        df = data2[(data2['wardname'] == ward) & (data2['lganame']== lga)]['wardcode']
        print(df)
        df1 = data2[(data2['wardname'] == ward) & (data2['lganame']== lga)]['geometry']
        if df.empty:
            continue
        cholera.at[i,'WardCode'] = df.item()
        cholera.loc[i,'geometry'] = df1.item()
    return cholera

In [113]:
df9 = get_ward_code(df8,df7)

6852    ABSAPR12
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
5699    42007
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
4553    42010
Name: wardcode, dtype: object
4668    42009
Name: wardcode, dtype: object
4473    42008
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
5620    42002
Name: wardcode, dtype: object
5702    42005
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
3752    10803
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name

Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
2481    70607
Name: wardcode, dtype: object
5269    BA1713
Name: wardcode, dtype: object
4220    80610
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
3832    80611
Name: wardcode, dtype: object
298    GMSGME04
Name: wardcode, dtype: object
5200    42908
Name: wardcode, dtype: object
5330    NISMKW06
Name: wardcode, dtype: object
3092    71210
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
1300    ADSGRE02
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
4557    BA1705
Name: wardcode, dtype: object
5528    NISMKW03
Name: wardcode, dtype: object
9156    KN3608
Name: wardcode, dtype: object
8740    GMSKLT01
Name: wardcode, dtype: object
7377    71408
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtyp

5646    JI2307
Name: wardcode, dtype: object
5261    43205
Name: wardcode, dtype: object
5451    43008
Name: wardcode, dtype: object
4844    43002
Name: wardcode, dtype: object
5058    43010
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
4024    70703
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
4458    40307
Name: wardcode, dtype: object
4465    40310
Name: wardcode, dtype: object
4631    40306
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
4887    43006
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
2489    81208
Name: wardcode, dtype: object
4500    43203
Name: wardcode, dtype: object
4469    43201
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
5359    41705
Name: wardcode, dtype: object
5415    42809
Name: wardcode, dtype: object
351

Series([], Name: wardcode, dtype: object)
6295    KD1106
Name: wardcode, dtype: object
5440    40704
Name: wardcode, dtype: object
3666    ODSOET07
Name: wardcode, dtype: object
4816    43103
Name: wardcode, dtype: object
9249    43106
Name: wardcode, dtype: object
870    ODSOWT07
Name: wardcode, dtype: object
3136    TRSGKA03
Name: wardcode, dtype: object
4566    BA1815
Name: wardcode, dtype: object
5288    BA1812
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
3386    ODSOET09
Name: wardcode, dtype: object
3378    ODSANW04
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
3427    ODSOET04
Name: wardcode, dtype: object
3740    KN2107
Name: wardcode, dtype: object
5529    81008
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
2867    41604
Name: wardcode, dtype: object
1914    TRSBBB05
Name: wardcode, dtype: object
9139    TRSWKR08
Name: wardcode, dtype: object
Series([], Name

Series([], Name: wardcode, dtype: object)
8229    BNSPKG08
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
1779    41601
Name: wardcode, dtype: object
6761    ABSUNC06
Name: wardcode, dtype: object
2723    CRSKAM06
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
6888    ABSUNC03
Name: wardcode, dtype: object
8655    CRSKMM07
Name: wardcode, dtype: object
3769    BNSPKG12
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
3806    51205
Name: wardcode, dtype: object
630    ABSABA12
Name: wardcode, dtype: object
7206    ABSUNC07
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
8736    CRSBNS10
Name: wardcode, dtype: object
3047    BNSPKG02
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
6800    ABSUNC08
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: w

In [114]:
df12 = df9[df9['WardCode']=='']

In [117]:
df12['ward_duplicate'] = df12['ward']

/var/folders/ks/cpcnjkt95qx6v7hynw8xydkr0000gp/T/ipykernel_3431/325954574.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12['ward_duplicate'] = df12['ward']


In [118]:
df12

,State,LGA,ward,WardCode,geometry,ward_duplicate
1,Ondo,Okitipupa,OkitipupaI 1,,,OkitipupaI 1
2,Ebonyi,Ohaukwu,Ngbo,,,Ngbo
3,Bayelsa,Southern Ijaw,Ekowe,,,Ekowe
4,Ondo,Irele,Akotogbo,,,Akotogbo
5,Katsina,Katsina,Gabas III,,,Gabas III
...,...,...,...,...,...,...
897,Ekiti,Moba,Erinmope,,,Erinmope
901,Zamfara,Birnin Magaji/Kiyaw,Birnin Magaji (Birnin,,,Birnin Magaji (Birnin
903,Zamfara,Maradun,Faru/Magami,,,Faru/Magami
905,Zamfara,Maradun,Damaga/Gama Giwa,,,Damaga/Gama Giwa


In [119]:
df12['ward'] = df12['ward'].apply(lambda x: difflib.get_close_matches(x, df7['wardname']))

/var/folders/ks/cpcnjkt95qx6v7hynw8xydkr0000gp/T/ipykernel_3431/2727019937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12['ward'] = df12['ward'].apply(lambda x: difflib.get_close_matches(x, df7['wardname']))


In [120]:
df12.reset_index(drop=True,inplace=True)

In [121]:
df12

,State,LGA,ward,WardCode,geometry,ward_duplicate
0,Ondo,Okitipupa,"[Okitipupa 1, Okitipupa 2]",,,OkitipupaI 1
1,Ebonyi,Ohaukwu,"[Ngo, Ngbo 1, Ogbo]",,,Ngbo
2,Bayelsa,Southern Ijaw,"[Ekowe, Ekwe 1, Ekwegbe 2]",,,Ekowe
3,Ondo,Irele,"[Akotogbo 2, Akotogbo 1, Arogbo 3]",,,Akotogbo
4,Katsina,Katsina,"[Gabas 3, Gabas 2, Gabas 1]",,,Gabas III
...,...,...,...,...,...,...
366,Ekiti,Moba,"[Erinmope 1, Erinmape 2, Erinmo Jesa]",,,Erinmope
367,Zamfara,Birnin Magaji/Kiyaw,"[Birnin Magaji, Birnin Magaji, Odeni Magaji / ...",,,Birnin Magaji (Birnin
368,Zamfara,Maradun,"[Faru Magami, Faragai, Magami]",,,Faru/Magami
369,Zamfara,Maradun,[Damaga Gamagiwa],,,Damaga/Gama Giwa


In [122]:
df12 = pd.read_csv('without(wardcode).csv')

In [123]:
df12

,Unnamed: 0,State,LGA,ward,Address,WardCode,geometry
0,0,Ondo,Okitipupa,Okitipupa 1,"Okitipupa,Ondo,Nigeria",NaN,NaN
1,1,Ebonyi,Ohaukwu,Ngbo 1,"Ohaukwu,Ebonyi,Nigeria",NaN,NaN
2,2,Bayelsa,Southern Ijaw,Ekowe,"Southern Ijaw,Bayelsa,Nigeria",NaN,NaN
3,3,Ondo,Irele,Akotogbo 1,"Irele,Ondo,Nigeria",NaN,NaN
4,4,Katsina,Katsina,Gabas 3,"Katsina,Katsina,Nigeria",NaN,NaN
...,...,...,...,...,...,...,...
235,362,Lagos,Shomolu,Igbobi / Fadeyi,"Shomolu,Lagos,Nigeria",NaN,NaN
236,365,Zamfara,Shinkafi,Jangeru,"Shinkafi,Zamfara,Nigeria",NaN,NaN
237,368,Zamfara,Maradun,Faru Magami,"Maradun,Zamfara,Nigeria",NaN,NaN
238,369,Zamfara,Maradun,Damaga Gamagiwa,"Maradun,Zamfara,Nigeria",NaN,NaN


In [124]:
df12 = df12.drop('Unnamed: 0',axis=1)

In [125]:
df12 = df12.reset_index(drop=True)

In [126]:
df7[df7['wardname']=='Ngbo 1']

,FID,globalid,uniq_id,timestamp,editor,wardname,wardcode,lganame,lgacode,statename,statecode,amapcode,status,source,urban,Shape__Are,Shape__Len,geometry
2417,2418,73412958-1808-45cc-8683-caaeca2e57a8,491077,2020-02-18,najib.adam,Ngbo 1,EBSHKW15,Ohaukwu,11012,Ebonyi,EB,NIE EBS HKW,Operational Placeholder,INEC,No,0.005843,0.522844,"POLYGON ((7.96523 6.45720, 7.96615 6.46704, 7...."


In [127]:
df9

,State,LGA,ward,WardCode,geometry
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((7.46374718028818 5.48372073212613, 7..."
1,Ondo,Okitipupa,OkitipupaI 1,,
2,Ebonyi,Ohaukwu,Ngbo,,
3,Bayelsa,Southern Ijaw,Ekowe,,
4,Ondo,Irele,Akotogbo,,
...,...,...,...,...,...
904,Benue,Agatu,Usha,BNSGTU10,"POLYGON ((7.86704295107137 7.89419374624447, 7..."
905,Zamfara,Maradun,Damaga/Gama Giwa,,
906,Benue,Agatu,Egba,BNSGTU01,"POLYGON ((7.98836411391278 7.86106937705017, 7..."
907,Adamawa,Girei,Girei A,,


In [128]:
df9 = df9[~(df9['WardCode']== "")]

In [129]:
df9

,State,LGA,ward,WardCode,geometry
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((7.46374718028818 5.48372073212613, 7..."
7,Katsina,Kankia,Rimaye,42007,"POLYGON ((7.87640474338423 12.235241306741, 7...."
10,Katsina,Kankia,Tsa Magam,42010,MULTIPOLYGON (((7.66813721685912 12.3379444635...
11,Katsina,Kankia,Tafashiya Nasarawa,42009,"POLYGON ((7.81081842403108 12.535974301273, 7...."
12,Katsina,Kankia,Sukuntuni,42008,"POLYGON ((7.79887369606428 12.2639991427463, 7..."
...,...,...,...,...,...
900,Lagos,Ajeromi/Ifelodun,Ajegunle,LASATO02,"POLYGON ((3.3509085676763 6.45495156086588, 3...."
902,Zamfara,Maru,Bingi,81002,"POLYGON ((6.37280000011998 12.0712999995953, 6..."
904,Benue,Agatu,Usha,BNSGTU10,"POLYGON ((7.86704295107137 7.89419374624447, 7..."
906,Benue,Agatu,Egba,BNSGTU01,"POLYGON ((7.98836411391278 7.86106937705017, 7..."


In [130]:
df12 = get_ward_code(df12,df7)

Series([], Name: wardcode, dtype: object)
2417    EBSHKW15
Name: wardcode, dtype: object
3208    BYSSPR05
Name: wardcode, dtype: object
3700    ODSIRL03
Name: wardcode, dtype: object
5167    42105
Name: wardcode, dtype: object
5547    42111
Name: wardcode, dtype: object
9034    CRSTGD02
Name: wardcode, dtype: object
2513    EBSAFK24
Name: wardcode, dtype: object
6113    EBSHKW35
Name: wardcode, dtype: object
9292    EBSHKW33
Name: wardcode, dtype: object
4656    KN0909
Name: wardcode, dtype: object
7069    BYSGBB11
Name: wardcode, dtype: object
1259    BYSBRS02
Name: wardcode, dtype: object
7220    BYSGBB03
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
1258    BYSBRS01
Name: wardcode, dtype: object
1256    BYSBRS04
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
Series([]

7441    BYSNEM06
Name: wardcode, dtype: object
5626    42107
Name: wardcode, dtype: object
6964    IMSNKR02
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
666    ANSAWK19
Name: wardcode, dtype: object
485    IMSRLU08
Name: wardcode, dtype: object
7018    IMSRLU03
Name: wardcode, dtype: object
8121    ABSHAF14
Name: wardcode, dtype: object
6822    IMSRLU06
Name: wardcode, dtype: object
8574    GMSYDB07
Name: wardcode, dtype: object
2819    BNSYGJ06
Name: wardcode, dtype: object
1414    IMSNKR03
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
Series([], Name: wardcode, dtype: object)
4126    50109
Name: wardcode, dtype: object
5676    42112
Name: wardcode, dtype: object
Series([], Name: wardcode, dtype: object)
4677    NASLFA07
Name: wardcode, dtype: object
8328    BNSJUX11
Name: wardcode, dtype: object
486    IMSAFR14
Name: wardcode, dtype: object
1777    IMSWER05
Name: wardcode, dtype: object
315

In [131]:
df12 = df12[~(df12['WardCode']=="")]

In [132]:
df7[df7['wardname']== 'Okitipupa 1']['wardcode']

3417    ODSPUP12
Name: wardcode, dtype: object

In [133]:
df7[df7['wardcode']=='42007']

,FID,globalid,uniq_id,timestamp,editor,wardname,wardcode,lganame,lgacode,statename,statecode,amapcode,status,source,urban,Shape__Are,Shape__Len,geometry
5699,5700,cc81a3e0-e121-4c50-aa6c-b662ef140dcb,492859,2019-03-16,oluseye.abraham,Rimaye,42007,Kankia,420,Katsina,KT,NIE KTS KNK RIM,Operational Validated,INEC,No,0.006775,0.55105,"POLYGON ((7.87640 12.23524, 7.86746 12.23394, ..."


In [134]:
df[df['LGA']=='Ohaukwu']

,State,LGA,ward
118,Ebonyi,Ohaukwu,NaN
119,Ebonyi,Ohaukwu,NaN
120,Ebonyi,Ohaukwu,NaN
121,Ebonyi,Ohaukwu,NaN
122,Ebonyi,Ohaukwu,NaN
...,...,...,...
222,Ebonyi,Ohaukwu,Okposhi Ward
224,Ebonyi,Ohaukwu,Okposhi Ward
248,Ebonyi,Ohaukwu,Ngbo Ward
249,Ebonyi,Ohaukwu,Umuogudu Oshia Ward


In [135]:
df9

,State,LGA,ward,WardCode,geometry
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((7.46374718028818 5.48372073212613, 7..."
7,Katsina,Kankia,Rimaye,42007,"POLYGON ((7.87640474338423 12.235241306741, 7...."
10,Katsina,Kankia,Tsa Magam,42010,MULTIPOLYGON (((7.66813721685912 12.3379444635...
11,Katsina,Kankia,Tafashiya Nasarawa,42009,"POLYGON ((7.81081842403108 12.535974301273, 7...."
12,Katsina,Kankia,Sukuntuni,42008,"POLYGON ((7.79887369606428 12.2639991427463, 7..."
...,...,...,...,...,...
900,Lagos,Ajeromi/Ifelodun,Ajegunle,LASATO02,"POLYGON ((3.3509085676763 6.45495156086588, 3...."
902,Zamfara,Maru,Bingi,81002,"POLYGON ((6.37280000011998 12.0712999995953, 6..."
904,Benue,Agatu,Usha,BNSGTU10,"POLYGON ((7.86704295107137 7.89419374624447, 7..."
906,Benue,Agatu,Egba,BNSGTU01,"POLYGON ((7.98836411391278 7.86106937705017, 7..."


In [137]:
df12

,State,LGA,ward,Address,WardCode,geometry
1,Ebonyi,Ohaukwu,Ngbo 1,"Ohaukwu,Ebonyi,Nigeria",EBSHKW15,"POLYGON ((7.96522790674254 6.45719851699801, 7..."
2,Bayelsa,Southern Ijaw,Ekowe,"Southern Ijaw,Bayelsa,Nigeria",BYSSPR05,"POLYGON ((6.09150254031005 4.74682749539267, 6..."
3,Ondo,Irele,Akotogbo 1,"Irele,Ondo,Nigeria",ODSIRL03,"POLYGON ((5.05317543230041 6.47487401012751, 5..."
4,Katsina,Katsina,Gabas 3,"Katsina,Katsina,Nigeria",42105,"POLYGON ((7.64238437284058 13.0548984508029, 7..."
5,Katsina,Katsina,Yamma 1,"Katsina,Katsina,Nigeria",42111,"POLYGON ((7.59157513868178 12.9966134240703, 7..."
...,...,...,...,...,...,...
235,Lagos,Shomolu,Igbobi / Fadeyi,"Shomolu,Lagos,Nigeria",LASSMU08,"POLYGON ((3.37455933094839 6.52792322257869, 3..."
236,Zamfara,Shinkafi,Jangeru,"Shinkafi,Zamfara,Nigeria",81102,"POLYGON ((6.45503879690301 12.9979582666607, 6..."
237,Zamfara,Maradun,Faru Magami,"Maradun,Zamfara,Nigeria",80903,"POLYGON ((6.40548843194563 12.6509678458894, 6..."
238,Zamfara,Maradun,Damaga Gamagiwa,"Maradun,Zamfara,Nigeria",80901,"POLYGON ((6.50083503460922 12.4465499294357, 6..."


In [138]:
df13 = pd.concat([df9,df12], axis=0)

In [139]:
df13 = df13.reset_index(drop=True)

In [140]:
df13.to_csv('Positive_datset.csv', index=False)

In [141]:
df13['Target'] =1

In [142]:
df14 = gpd.read_file('Negative_dataset.shp')

In [143]:
df14 = df14.to_crs('epsg:26332')

In [144]:
df14.crs

<Projected CRS: EPSG:26332>
Name: Minna / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Nigeria - offshore deep water - east of 6°E.
- bounds: (6.0, 2.61, 7.82, 3.68)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: Minna
- Ellipsoid: Clarke 1880 (RGS)
- Prime Meridian: Greenwich

In [147]:
df13 = gpd.GeoDataFrame(df13, geometry='geometry',)

In [148]:
df13

,State,LGA,ward,WardCode,geometry,Address,Target
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((7.46375 5.48372, 7.46396 5.47503, 7....",NaN,1
1,Katsina,Kankia,Rimaye,42007,"POLYGON ((7.87640 12.23524, 7.86746 12.23394, ...",NaN,1
2,Katsina,Kankia,Tsa Magam,42010,"MULTIPOLYGON (((7.66814 12.33794, 7.66313 12.3...",NaN,1
3,Katsina,Kankia,Tafashiya Nasarawa,42009,"POLYGON ((7.81082 12.53597, 7.81671 12.53810, ...",NaN,1
4,Katsina,Kankia,Sukuntuni,42008,"POLYGON ((7.79887 12.26400, 7.78990 12.26946, ...",NaN,1
...,...,...,...,...,...,...,...
708,Lagos,Shomolu,Igbobi / Fadeyi,LASSMU08,"POLYGON ((3.37456 6.52792, 3.37455 6.52787, 3....","Shomolu,Lagos,Nigeria",1
709,Zamfara,Shinkafi,Jangeru,81102,"POLYGON ((6.45504 12.99796, 6.43705 13.02863, ...","Shinkafi,Zamfara,Nigeria",1
710,Zamfara,Maradun,Faru Magami,80903,"POLYGON ((6.40549 12.65097, 6.40956 12.64589, ...","Maradun,Zamfara,Nigeria",1
711,Zamfara,Maradun,Damaga Gamagiwa,80901,"POLYGON ((6.50084 12.44655, 6.50026 12.44436, ...","Maradun,Zamfara,Nigeria",1


In [149]:
df13 = df13.set_crs('epsg:26332')

In [150]:
df14 = df14[['statename','lganame','wardname','wardcode','geometry']]

In [151]:
df13 = df13.drop('Address',axis=1)

In [152]:
df14.columns = ['State','LGA','ward','WardCode','geometry']

In [153]:
df14['Target'] = 0

In [650]:
df14

,State,LGA,ward,WardCode,geometry,Target
0,Akwa Ibom,Ikot Ekpene,Ikot Inyang / Ikot Ekpene 6,AKSKTE10,"POLYGON ((354467.524 571522.057, 354262.062 57...",0
1,Akwa Ibom,Ikot Ekpene,Uruaobo / Ikot Ekpene 5,AKSKTE11,"POLYGON ((360376.632 573332.619, 360078.374 57...",0
2,Imo,Nwangele,Amaigbo 2,IMSAMG07,"POLYGON ((287235.135 627809.792, 287054.365 62...",0
3,Abia,Umuahia North,Afugiri 1,ABSUMA02,"POLYGON ((330659.351 620853.549, 331144.592 62...",0
4,Imo,Isiala Mbano,Ogbor,IMSUML08,"POLYGON ((300430.458 623092.277, 300492.130 62...",0
...,...,...,...,...,...,...
323,Ogun,Yewa North,Ijoun,OGSYEN07,"POLYGON ((-190197.737 813910.881, -189775.363 ...",0
324,Ogun,Yewa North,Igbogila / Ibese,OGSYEN06,"POLYGON ((-158152.075 771693.527, -158937.853 ...",0
325,Ogun,Yewa North,Sunwa,OGSYEN11,"POLYGON ((-169938.024 792979.349, -171073.916 ...",0
326,Ogun,Imeko Afon,Imeko,OGSION07,"POLYGON ((-177263.515 833753.722, -176708.544 ...",0


In [651]:
df13

,State,LGA,ward,WardCode,geometry,Target
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((329878.126 606239.826, 329899.087 60...",1
1,Katsina,Kankia,Rimaye,42007,"POLYGON ((377863.410 1352701.667, 376889.942 1...",1
2,Katsina,Kankia,Tsa Magam,42010,"MULTIPOLYGON (((355262.803 1364164.364, 354719...",1
3,Katsina,Kankia,Tafashiya Nasarawa,42009,"POLYGON ((370876.737 1385994.233, 371517.595 1...",1
4,Katsina,Kankia,Sukuntuni,42008,"POLYGON ((369443.373 1355918.626, 368469.446 1...",1
...,...,...,...,...,...,...
708,Lagos,Shomolu,Igbobi / Fadeyi,LASSMU08,"POLYGON ((-122845.713 724927.158, -122846.791 ...",1
709,Zamfara,Shinkafi,Jangeru,81102,"POLYGON ((224013.125 1438171.160, 222095.374 1...",1
710,Zamfara,Maradun,Faru Magami,80903,"POLYGON ((218248.548 1399817.751, 218685.369 1...",1
711,Zamfara,Maradun,Damaga Gamagiwa,80901,"POLYGON ((228397.116 1377092.070, 228332.183 1...",1


In [652]:
df15 = pd.concat([df13,df14], axis=0)

In [654]:
df15.to_csv('complete_dataset.csv')

In [ ]:
def get_number_of_public_water_supplies(cholera, water):
    cholera = cholera.copy()
    cholera['Number_of_public_water_supply'] = 0
    size = cholera.shape[0]
    for i in range(0,size):
        lga = cholera.loc[i,'WardCode']
        df = water[water['ward_code'] == lga].shape[0]
        cholera.at[i,'Number_of_public_water_supply'] = df
    return cholera

In [663]:
df15.reset_index(drop=True, inplace=True)

In [665]:
df15 = get_number_of_public_water_supplies(df15,df3)

In [666]:
df15

,State,LGA,ward,WardCode,geometry,Target,Number_of_public_water_supply
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((329878.126 606239.826, 329899.087 60...",1,17
1,Katsina,Kankia,Rimaye,42007,"POLYGON ((377863.410 1352701.667, 376889.942 1...",1,0
2,Katsina,Kankia,Tsa Magam,42010,"MULTIPOLYGON (((355262.803 1364164.364, 354719...",1,0
3,Katsina,Kankia,Tafashiya Nasarawa,42009,"POLYGON ((370876.737 1385994.233, 371517.595 1...",1,0
4,Katsina,Kankia,Sukuntuni,42008,"POLYGON ((369443.373 1355918.626, 368469.446 1...",1,0
...,...,...,...,...,...,...,...
1036,Ogun,Yewa North,Ijoun,OGSYEN07,"POLYGON ((-190197.737 813910.881, -189775.363 ...",0,7
1037,Ogun,Yewa North,Igbogila / Ibese,OGSYEN06,"POLYGON ((-158152.075 771693.527, -158937.853 ...",0,1
1038,Ogun,Yewa North,Sunwa,OGSYEN11,"POLYGON ((-169938.024 792979.349, -171073.916 ...",0,4
1039,Ogun,Imeko Afon,Imeko,OGSION07,"POLYGON ((-177263.515 833753.722, -176708.544 ...",0,14


In [668]:
def get_number_of_dump_sites(cholera, water):
    cholera = cholera.copy()
    cholera['Number_of_dump_sites'] = 0
    size = cholera.shape[0]
    for i in range(0,size):
        lga = cholera.loc[i,'WardCode']
        df = water[water['ward_code'] == lga].shape[0]
        cholera.at[i,'Number_of_dump_sites'] = df
    return cholera

In [669]:
df4 

,id,global_id,name,type,ownership,ward_code,category,timestamp,lga_name,state_code,state_name,geometry
0,40072,bdcec9cc-06e5-47bb-b30f-a31e0ff8232d,Dumping Site Umuigaga Community,Unofficial,NaN,EBSNCA06,Mixed Dumpsite,2018-02-02 07:43:42.757000+00:00,Onicha,EB,Ebonyi,POINT (373318.148 676490.734)
1,40073,af3a54eb-287c-45c0-b07d-b778efa21fc3,Dumping Site Ndiagu Umuchita Community,Unofficial,NaN,EBSNCA16,Mixed Dumpsite,2018-02-02 07:43:42.757000+00:00,Onicha,EB,Ebonyi,POINT (373653.843 676145.056)
2,40074,84aafd99-3ae1-4453-9361-f790734b6079,Ndiagu Ufueze Dumpsite,Unofficial,NaN,EBSNCA16,Mixed Dumpsite,2018-02-02 07:43:42.757000+00:00,Onicha,EB,Ebonyi,POINT (373390.025 676290.642)
3,40075,ab43c257-2ca9-4621-975b-c705bcbf3736,Itawure Dumpsite 1,Unofficial,NaN,EKSEFO07,Organic Dumpsite,2018-02-02 07:43:42.757000+00:00,Efon,EK,Ekiti,POINT (53089.107 855940.826)
4,40076,62db3e88-f198-4849-9dfe-78faab3e0d14,Itawure Dumpsite 2,Unofficial,NaN,EKSEFO07,Organic Dumpsite,2018-02-02 07:43:42.757000+00:00,Efon,EK,Ekiti,POINT (53140.932 856101.035)
...,...,...,...,...,...,...,...,...,...,...,...,...
6472,40067,43bf8b68-cf63-4af4-bb4b-4f5036098b88,Irabiji Ibokun Road Egbeda,Unofficial,NaN,OSSOLA08,Organic Dumpsite,2020-05-13 11:59:07.463000+00:00,Olorunda,OS,Osun,POINT (20602.440 864349.912)
6473,40068,4f7ab7a4-2613-4640-ad77-bfecc44d17f2,Fotab Road Dump Site,Unofficial,NaN,OSSIET04,Mixed Dumpsite,2020-05-13 11:59:07.463000+00:00,Ife East,OS,Osun,POINT (4018.088 831052.983)
6474,40069,329df79f-4a77-45db-ac77-c06356072caa,Ajebamidele Dumpsite,Unofficial,NaN,OSSIET04,Mixed Dumpsite,2020-05-13 11:59:07.463000+00:00,Ife East,OS,Osun,POINT (3821.142 831057.958)
6475,40070,6b998ae5-3ce2-4419-86bf-7fcddd46ff94,Ifedebi Dumpsite,Unofficial,NaN,OSSIET09,Mixed Dumpsite,2020-05-13 11:59:07.463000+00:00,Ife East,OS,Osun,POINT (10954.069 823541.453)


In [676]:
df15 = get_number_of_dump_sites(df15, df4)

In [672]:
def get_number_of_markets(cholera, water):
    cholera = cholera.copy()
    cholera['Number_of_markets'] = 0
    size = cholera.shape[0]
    for i in range(0,size):
        lga = cholera.loc[i,'WardCode']
        df = water[water['ward_code'] == lga].shape[0]
        cholera.at[i,'Number_of_markets'] = df
    return cholera

In [674]:
df15 = get_number_of_markets(df15,df6)

In [678]:
df15['area'] = df15['geometry'].area

In [679]:
df15

,State,LGA,ward,WardCode,geometry,Target,Number_of_public_water_supply,Number_of_markets,Number_of_dump_sites,area
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((329878.126 606239.826, 329899.087 60...",1,17,5,3,2.495595e+07
1,Katsina,Kankia,Rimaye,42007,"POLYGON ((377863.410 1352701.667, 376889.942 1...",1,0,1,0,8.149431e+07
2,Katsina,Kankia,Tsa Magam,42010,"MULTIPOLYGON (((355262.803 1364164.364, 354719...",1,0,0,0,1.193838e+08
3,Katsina,Kankia,Tafashiya Nasarawa,42009,"POLYGON ((370876.737 1385994.233, 371517.595 1...",1,0,0,0,6.914710e+07
4,Katsina,Kankia,Sukuntuni,42008,"POLYGON ((369443.373 1355918.626, 368469.446 1...",1,0,0,0,6.767949e+07
...,...,...,...,...,...,...,...,...,...,...
1036,Ogun,Yewa North,Ijoun,OGSYEN07,"POLYGON ((-190197.737 813910.881, -189775.363 ...",0,7,2,8,3.896691e+08
1037,Ogun,Yewa North,Igbogila / Ibese,OGSYEN06,"POLYGON ((-158152.075 771693.527, -158937.853 ...",0,1,0,3,1.623994e+08
1038,Ogun,Yewa North,Sunwa,OGSYEN11,"POLYGON ((-169938.024 792979.349, -171073.916 ...",0,4,3,15,2.698987e+08
1039,Ogun,Imeko Afon,Imeko,OGSION07,"POLYGON ((-177263.515 833753.722, -176708.544 ...",0,14,2,19,1.005402e+08


In [692]:
def get_average_distance(cholera,data1,data2,title):
    cholera = cholera.copy()
    cholera[title] = 0
    size = cholera.shape[0]
    for i in range(0,size):
        lga = cholera.loc[i,'WardCode']
        dataset2 = data2[data2['ward_code'] == lga]['geometry'].reset_index(drop=True)
        dataset1 = data1[data1['ward_code'] == lga]['geometry'].reset_index(drop=True)
        total_distance = 0
        n1 = len(dataset1)
        n2 = len(dataset2)
        print(n1)
        if n1 == 0 or n2 == 0:
            continue
        for k in range(0,n1):
            for j in range(0,n2):
                total_distance += dataset1[k].distance(datset2[j])
        average_distance = total_distance / (n1 * n2)
        cholera.at[i,title] = average_distance
    return cholera

In [684]:
df15 = get_average_distance(df15,df4,df3)

3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
0
0
0
0
1
0
0
0
15
0
4
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
7
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
2
0
0
0
0
0
0
6
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
1
1
0
0
0
0
12
0
0
0
0
0
2
2
5
0
2
0
0
0
0
0
0
0
12
0
0
2
1
64
0
2
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
2
3
0
0
0
0
0
1
0
0
0
0
4
0
0
0
0
0
0
0


In [690]:
df15 = get_average_distance(df15,df6,df4)

5
1
0
0
0
0
1
3
0
1
0
0
2
0
0
0
0
0
0
3
1
1
3
0
1
1
0
0
1
10
0
3
0
1
2
1
0
1
5
1
2
0
0
0
2
2
0
4
1
0
2
1
3
1
1
0
0
2
5
0
0
0
0
0
1
0
1
1
1
1
0
0
0
0
0
2
1
3
1
1
2
1
2
1
2
2
0
0
1
0
0
3
1
0
1
1
0
0
0
0
0
2
0
1
1
1
2
2
1
3
0
0
0
0
2
0
1
1
0
0
1
0
1
0
1
0
2
1
1
1
0
1
0
0
2
0
0
0
0
1
2
0
0
2
1
2
0
0
2
1
0
0
3
1
4
0
0
0
0
0
1
0
0
0
0
1
3
1
1
1
0
0
0
0
0
0
0
1
1
0
1
1
2
1
5
0
1
1
0
1
3
0
0
0
0
0
0
0
0
1
0
0
0
1
1
1
0
1
0
0
3
1
0
0
0
0
1
0
0
1
0
1
0
1
1
0
3
1
0
1
0
1
1
0
0
0
0
0
2
0
0
0
0
1
1
0
1
0
0
1
0
0
0
2
4
2
2
1
0
0
1
1
0
0
2
0
0
0
1
0
1
0
0
1
2
9
0
0
1
1
2
0
2
1
0
0
0
2
0
1
1
0
0
0
2
0
1
3
2
0
0
0
2
1
0
0
1
1
0
8
0
0
0
1
0
0
0
0
1
0
0
0
3
2
1
1
1
0
1
0
1
1
0
1
0
0
0
0
1
0
0
1
3
1
0
0
0
3
0
2
1
1
2
1
2
0
0
1
0
0
0
1
1
2
3
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
1
1
1
1
0
0
3
1
0
0
0
1
0
6
4
0
0
1
5
1
2
0
1
0
5
0
1
1
1
0
1
1
0
4
0
0
0
0
1
0
1
1
8
1
0
1
2
0
0
0
6
4
0
3
0
2
3
0
1
1
0
1
0
0
2
1
1
1
2
2
2
0
0
0
0
2
0
1
2
4
1
3
2
0
0
8
6
0
3
3
2
7
2
2
3
2
8
2
1
0
1
1
0
1
0
1
5
2
1
0
1
1
2
4
0
3
1
2
1
1

In [694]:
df15 = get_average_distance(df15,df6,df3)

5
1
0
0
0
0
1
3
0
1
0
0
2
0
0
0
0
0
0
3
1
1
3
0
1
1
0
0
1
10
0
3
0
1
2
1
0
1
5
1
2
0
0
0
2
2
0
4
1
0
2
1
3
1
1
0
0
2
5
0
0
0
0
0
1
0
1
1
1
1
0
0
0
0
0
2
1
3
1
1
2
1
2
1
2
2
0
0
1
0
0
3
1
0
1
1
0
0
0
0
0
2
0
1
1
1
2
2
1
3
0
0
0
0
2
0
1
1
0
0
1
0
1
0
1
0
2
1
1
1
0
1
0
0
2
0
0
0
0
1
2
0
0
2
1
2
0
0
2
1
0
0
3
1
4
0
0
0
0
0
1
0
0
0
0
1
3
1
1
1
0
0
0
0
0
0
0
1
1
0
1
1
2
1
5
0
1
1
0
1
3
0
0
0
0
0
0
0
0
1
0
0
0
1
1
1
0
1
0
0
3
1
0
0
0
0
1
0
0
1
0
1
0
1
1
0
3
1
0
1
0
1
1
0
0
0
0
0
2
0
0
0
0
1
1
0
1
0
0
1
0
0
0
2
4
2
2
1
0
0
1
1
0
0
2
0
0
0
1
0
1
0
0
1
2
9
0
0
1
1
2
0
2
1
0
0
0
2
0
1
1
0
0
0
2
0
1
3
2
0
0
0
2
1
0
0
1
1
0
8
0
0
0
1
0
0
0
0
1
0
0
0
3
2
1
1
1
0
1
0
1
1
0
1
0
0
0
0
1
0
0
1
3
1
0
0
0
3
0
2
1
1
2
1
2
0
0
1
0
0
0
1
1
2
3
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
1
1
1
1
0
0
3
1
0
0
0
1
0
6
4
0
0
1
5
1
2
0
1
0
5
0
1
1
1
0
1
1
0
4
0
0
0
0
1
0
1
1
8
1
0
1
2
0
0
0
6
4
0
3
0
2
3
0
1
1
0
1
0
0
2
1
1
1
2
2
2
0
0
0
0
2
0
1
2
4
1
3
2
0
0
8
6
0
3
3
2
7
2
2
3
2
8
2
1
0
1
1
0
1
0
1
5
2
1
0
1
1
2
4
0
3
1
2
1
1

In [695]:
df15.to_csv('ultimate_complete_dataset.csv')

In [696]:
df15

,State,LGA,ward,WardCode,geometry,Target,Number_of_public_water_supply,Number_of_markets,Number_of_dump_sites,area,avg_distance_btw_dumpsites_&_water_sources,avg_distance_btw_markets_&_dumpsites,avg_distance_btw_markets_&_water_sources
0,Abia,Umuahia South,Ubakala A,ABSAPR12,"POLYGON ((329878.126 606239.826, 329899.087 60...",1,17,5,3,2.495595e+07,2228.902305,686355.433372,688063.337370
1,Katsina,Kankia,Rimaye,42007,"POLYGON ((377863.410 1352701.667, 376889.942 1...",1,0,1,0,8.149431e+07,0.000000,0.000000,0.000000
2,Katsina,Kankia,Tsa Magam,42010,"MULTIPOLYGON (((355262.803 1364164.364, 354719...",1,0,0,0,1.193838e+08,0.000000,0.000000,0.000000
3,Katsina,Kankia,Tafashiya Nasarawa,42009,"POLYGON ((370876.737 1385994.233, 371517.595 1...",1,0,0,0,6.914710e+07,0.000000,0.000000,0.000000
4,Katsina,Kankia,Sukuntuni,42008,"POLYGON ((369443.373 1355918.626, 368469.446 1...",1,0,0,0,6.767949e+07,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,Ogun,Yewa North,Ijoun,OGSYEN07,"POLYGON ((-190197.737 813910.881, -189775.363 ...",0,7,2,8,3.896691e+08,12258.199760,827874.734589,825165.705322
1037,Ogun,Yewa North,Igbogila / Ibese,OGSYEN06,"POLYGON ((-158152.075 771693.527, -158937.853 ...",0,1,0,3,1.623994e+08,12232.806922,0.000000,0.000000
1038,Ogun,Yewa North,Sunwa,OGSYEN11,"POLYGON ((-169938.024 792979.349, -171073.916 ...",0,4,3,15,2.698987e+08,12070.320915,818847.081765,812723.516118
1039,Ogun,Imeko Afon,Imeko,OGSION07,"POLYGON ((-177263.515 833753.722, -176708.544 ...",0,14,2,19,1.005402e+08,2369.499677,847787.535338,847452.264997
